<a href="https://colab.research.google.com/github/Veronika2311/ML_CL_HSE/blob/main/HW_class2_Regression_Texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На этом занятии мы попробуем задачу регрессии. Данные возьмем вот эти - https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Перед обучением обучением модели, нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы


In [132]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from nltk.corpus import stopwords

In [133]:
import gdown
!gdown 1VCFHZJii3D_XV-rGhu2c_b9M7LECLup9

Downloading...
From: https://drive.google.com/uc?id=1VCFHZJii3D_XV-rGhu2c_b9M7LECLup9
To: /content/IMDB-Movie-Data.csv
100% 310k/310k [00:00<00:00, 21.7MB/s]


In [134]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


## Что делать с NaN?
Есть 3 варианта

In [135]:
# 1. Убрать строки с NaN
print(data.isna().any())
data.shape

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)     True
Metascore              True
dtype: bool


(1000, 12)

In [136]:
print(data.shape)
tmp = data.dropna()
tmp.shape

(1000, 12)


(838, 12)

In [137]:
# 2. Превратить NaN в 0
print(data.shape)
tmp = data.fillna(0)
print(tmp.shape)

(1000, 12)
(1000, 12)


In [138]:
# 3. Превратить NaN в средние значения по колонке

# вычисляем средние для колонок с пустыми значениями
meta_mean = data['Metascore'].mean()
rev_mean = data['Revenue (Millions)'].mean()

#заменяем пустоты на средние значения
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

# проверяем присутствие NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

## Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет **целевой переменной, или таргетом** (y)<br>
Остальных данные будут **обучающей выборкой** (X)

In [139]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
995    A tight-knit team of rising investigators, alo...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
998    A pair of friends embark on a mission to reuni...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 1000, dtype: object

In [ ]:
# подготовим описания фильмов
data["text"] = data.Description.apply(lambda x: x.lower().split())

In [140]:
#очистка от стоп-слов
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words("english"))
preproc_texts = []

for i in data["text"]:
    text = []
    for word in i:
        if str(word) not in stop_words:
            text.append(word)
    preproc_texts.append(text)

data["text"] = preproc_texts

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
data["text"]

0      [group, intergalactic, criminals, forced, work...
1      [following, clues, origin, mankind,, team, fin...
2      [three, girls, kidnapped, man, diagnosed, 23, ...
3      [city, humanoid, animals,, hustling, theater, ...
4      [secret, government, agency, recruits, dangero...
                             ...                        
995    [tight-knit, team, rising, investigators,, alo...
996    [three, american, college, students, studying,...
997    [romantic, sparks, occur, two, dance, students...
998    [pair, friends, embark, mission, reunite, pal,...
999    [stuffy, businessman, finds, trapped, inside, ...
Name: text, Length: 1000, dtype: object

In [142]:
data.text.values[:5]

array([list(['group', 'intergalactic', 'criminals', 'forced', 'work', 'together', 'stop', 'fanatical', 'warrior', 'taking', 'control', 'universe.']),
       list(['following', 'clues', 'origin', 'mankind,', 'team', 'finds', 'structure', 'distant', 'moon,', 'soon', 'realize', 'alone.']),
       list(['three', 'girls', 'kidnapped', 'man', 'diagnosed', '23', 'distinct', 'personalities.', 'must', 'try', 'escape', 'apparent', 'emergence', 'frightful', 'new', '24th.']),
       list(['city', 'humanoid', 'animals,', 'hustling', 'theater', "impresario's", 'attempt', 'save', 'theater', 'singing', 'competition', 'becomes', 'grander', 'anticipates', 'even', "finalists'", 'find', 'lives', 'never', 'same.']),
       list(['secret', 'government', 'agency', 'recruits', 'dangerous', 'incarcerated', 'super-villains', 'form', 'defensive', 'task', 'force.', 'first', 'mission:', 'save', 'world', 'apocalypse.'])],
      dtype=object)

In [143]:
input_text = list(data.text.values)

In [144]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

[TaggedDocument(words=['adventures', 'writer', 'newt', 'scamander', 'new', "york's", 'secret', 'community', 'witches', 'wizards', 'seventy', 'years', 'harry', 'potter', 'reads', 'book', 'school.'], tags=[10]),
 TaggedDocument(words=['story', 'team', 'female', 'african-american', 'mathematicians', 'served', 'vital', 'role', 'nasa', 'early', 'years', 'u.s.', 'space', 'program.'], tags=[11])]

обучаем модель на текстах описаний фильмов (можно поизменять параметры)

In [145]:
#изменила параметры модели
model = Doc2Vec(documents, vector_size=10, window=4, min_count=1, workers=8)

In [146]:
model.save("D2V.model") # сохранение модели

In [147]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]


array([-0.05236586, -0.05975834, -0.09778433,  0.08829936,  0.03731123,
        0.00511391, -0.09554334, -0.05075314, -0.10293548,  0.02190863],
      dtype=float32)

Теперь нужно добавить векторы в датасет с остальными параметрами

In [148]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model.dv[x.tags][0])
    vectors.append(vec)

In [149]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v'+str(i) for i in range(1,11)])

split_df


,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,-0.052366,-0.059758,-0.097784,0.088299,0.037311,0.005114,-0.095543,-0.050753,-0.102935,0.021909
1,0.032020,0.047205,-0.039074,-0.029797,-0.028218,-0.084016,0.024819,0.092861,-0.101163,-0.037055
2,-0.036532,0.028637,-0.055364,0.028926,0.060798,-0.077613,-0.081259,-0.097972,0.042140,-0.092174
3,0.062431,0.072310,-0.059182,-0.040732,-0.008796,0.022401,-0.008776,-0.084796,-0.048371,0.014637
4,-0.015053,-0.072018,0.046644,-0.085173,0.030440,-0.043319,0.012275,-0.019983,0.043011,-0.084824
...,...,...,...,...,...,...,...,...,...,...
995,-0.061901,-0.098444,0.095745,0.099105,-0.020625,0.086851,0.100314,-0.098062,0.074733,-0.094351
996,0.089618,-0.034962,-0.034127,-0.032942,0.029193,-0.017745,-0.075185,-0.075066,-0.098512,-0.095409
997,-0.024297,0.093265,0.022026,-0.007757,0.059960,0.069314,-0.025209,-0.072907,-0.102488,0.005501
998,0.003623,0.017906,0.000390,-0.060346,-0.093811,0.094992,-0.045154,-0.094413,0.089744,0.056452


In [150]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(1000, 23)

In [151]:
result

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,-0.052366,-0.059758,-0.097784,0.088299,0.037311,0.005114,-0.095543,-0.050753,-0.102935,0.021909
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0.032020,0.047205,-0.039074,-0.029797,-0.028218,-0.084016,0.024819,0.092861,-0.101163,-0.037055
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,-0.036532,0.028637,-0.055364,0.028926,0.060798,-0.077613,-0.081259,-0.097972,0.042140,-0.092174
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0.062431,0.072310,-0.059182,-0.040732,-0.008796,0.022401,-0.008776,-0.084796,-0.048371,0.014637
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,-0.015053,-0.072018,0.046644,-0.085173,0.030440,-0.043319,0.012275,-0.019983,0.043011,-0.084824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,-0.061901,-0.098444,0.095745,0.099105,-0.020625,0.086851,0.100314,-0.098062,0.074733,-0.094351
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,0.089618,-0.034962,-0.034127,-0.032942,0.029193,-0.017745,-0.075185,-0.075066,-0.098512,-0.095409
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,-0.024297,0.093265,0.022026,-0.007757,0.059960,0.069314,-0.025209,-0.072907,-0.102488,0.005501
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0.003623,0.017906,0.000390,-0.060346,-0.093811,0.094992,-0.045154,-0.094413,0.089744,0.056452


In [152]:
# переопределим датасет, оставив только важное
vects = ['v'+str(i) for i in range(1,11)]
print(vects)
data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10']
              ]


data_sm.head(3)

['v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10']


,Runtime (Minutes),Year,Rating,Votes,Revenue (Millions),Metascore,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10
0,121,2014,8.1,757074,333.13,76.0,-0.052366,-0.059758,-0.097784,0.088299,0.037311,0.005114,-0.095543,-0.050753,-0.102935,0.021909
1,124,2012,7.0,485820,126.46,65.0,0.032020,0.047205,-0.039074,-0.029797,-0.028218,-0.084016,0.024819,0.092861,-0.101163,-0.037055
2,117,2016,7.3,157606,138.12,62.0,-0.036532,0.028637,-0.055364,0.028926,0.060798,-0.077613,-0.081259,-0.097972,0.042140,-0.092174


## Подготавливаем матрицы

In [153]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
        -5.07531352e-02, -1.02935478e-01,  2.19086297e-02],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
         9.28605050e-02, -1.01163194e-01, -3.70550826e-02],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
        -9.79721993e-02,  4.21404541e-02, -9.21742618e-02],
       ...,
       [ 9.80000000e+01,  2.00800000e+03,  7.06990000e+04, ...,
        -7.29068965e-02, -1.02488168e-01,  5.50131639e-03],
       [ 9.30000000e+01,  2.01400000e+03,  4.88100000e+03, ...,
        -9.44128335e-02,  8.97443667e-02,  5.64524010e-02],
       [ 8.70000000e+01,  2.01600000e+03,  1.24350000e+04, ...,
         4.31879871e-02,  7.77637493e-03,  6.39862940e-02]])

(1000, 15)

In [154]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
v6                    False
v7                    False
v8                    False
v9                    False
v10                   False
dtype: bool

In [155]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

(1000,)

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения.
Для этого используется StandardScaler.

До нормализации:

In [156]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 -0.05236586183309555,
 -0.05975833907723427,
 -0.09778432548046112,
 0.08829936385154724,
 0.03731123358011246,
 0.00511390995234251,
 -0.09554333984851837,
 -0.05075313523411751,
 -0.10293547809123993,
 0.021908629685640335]

In [158]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [159]:

list(sc.fit_transform(X)[0])

[0.4163497512303056,
 0.37979525138136244,
 3.1126899627963738,
 2.5961363010556906,
 1.0233613578368184,
 -0.9743582271503536,
 -1.093418920239567,
 -1.7544000282965124,
 1.3945945607895156,
 0.5656411966008059,
 -0.0146462752305986,
 -1.7049259822751213,
 -0.809841389862479,
 -1.5733827482338685,
 0.39670580152198587]

теперь с данными удобнее работать и обучать

In [160]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor = Ridge()


# обучаем
regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
print('MAE')
print(mean_absolute_error(y_test, y_preds))
print('MSE')
print(mean_squared_error(y_test, y_preds, squared=True))
print('RMSE')
print(mean_squared_error(y_test, y_preds, squared=False))

MAE
0.48713261188646473
MSE
0.4635001334699899
RMSE
0.6808084410977804


In [161]:
regressor = Ridge(0.01)


regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
print('MAE')
print(mean_absolute_error(y_test, y_preds))
print('MSE')
print(mean_squared_error(y_test, y_preds, squared=True))
print('RMSE')
print(mean_squared_error(y_test, y_preds, squared=False))

MAE
0.48711936896349395
MSE
0.46343420411841657
RMSE
0.680760019477067


In [162]:
regressor = LinearRegression()


regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
print('MAE')
print(mean_absolute_error(y_test, y_preds))
print('MSE')
print(mean_squared_error(y_test, y_preds, squared=True))
print('RMSE')
print(mean_squared_error(y_test, y_preds, squared=False))

MAE
0.48711923317761685
MSE
0.4634335441733931
RMSE
0.6807595347649514


In [164]:
regressor = Lasso()
#всё плохо, ошибки большие

regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
print('MAE')
print(mean_absolute_error(y_test, y_preds))
print('MSE')
print(mean_squared_error(y_test, y_preds, squared=True))
print('RMSE')
print(mean_squared_error(y_test, y_preds, squared=False))

MAE
0.7736000000000001
MSE
0.9734313244444442
RMSE
0.9866262334057636


In [165]:
regressor = Lasso(0.01)
#а тут уже ничего так

regressor.fit(X_train, y_train)
y_preds = regressor.predict(X_test)
print('MAE')
print(mean_absolute_error(y_test, y_preds))
print('MSE')
print(mean_squared_error(y_test, y_preds, squared=True))
print('RMSE')
print(mean_squared_error(y_test, y_preds, squared=False))

MAE
0.4848179741227267
MSE
0.46287301540598036
RMSE
0.6803477165435189


## Результаты
Я очистила данные от стопа-слов и варьировала параметр длинs вектора в doc2vec от 5 до 10, также я варьировала альфу и прочие параметры у регрессоров, но с параметрами по умолчанию ошибка получилась меньше (см.выше), а удаление стоп-слов не повлияло значимо на результат

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*